In [ ]:

!pip install ultralytics -q

import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print(" Warning: Using CPU")

from google.colab import drive
drive.mount('/content/drive')


In [ ]:

!unzip "/content/drive/MyDrive/volley_player_dataset.zip" -d /content/player_dataset

!cat /content/player_dataset/data.yaml

!yolo task=detect mode=train model=yolov8n.pt \
  data=/content/player_dataset/data.yaml \
  epochs=20 imgsz=640

!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt \
  source=/content/player_dataset/valid/images save=True


Archive:  /content/drive/MyDrive/volley_player_dataset.zip
 extracting: /content/player_dataset/README.dataset.txt  
 extracting: /content/player_dataset/README.roboflow.txt  
 extracting: /content/player_dataset/data.yaml  
   creating: /content/player_dataset/test/
   creating: /content/player_dataset/test/images/
 extracting: /content/player_dataset/test/images/Y2Mate_is-Professional-volleyball-game-produced-by-the-MOVE-N-SEE-Wide-Angle-camera_-See-description-below_-_1IySovqErQ-720p-1654995342377_mp4-21_jpg.rf.911470866b32e9e1cd22109fa66ba7d4.jpg  
 extracting: /content/player_dataset/test/images/rally_women_mp4-0_jpg.rf.d3e7ca6e7924320c89cb413263439e15.jpg  
 extracting: /content/player_dataset/test/images/rally_women_mp4-5_jpg.rf.f47d1057723df439737420de079e3d28.jpg  
 extracting: /content/player_dataset/test/images/volley-138-_png.rf.955759b02420e5cc7e3f70243925de5d.jpg  
 extracting: /content/player_dataset/test/images/volley-143-_png.rf.64d9029d34758ea027aaff6c76cddb60.jpg  
 

In [ ]:

!unzip "/content/drive/MyDrive/volley_ball_dataset.zip" -d /content/ball_dataset

!cat /content/ball_dataset/data.yaml

!yolo task=detect mode=train model=yolov8n.pt \
  data=/content/ball_dataset/data.yaml \
  epochs=20 imgsz=640

!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt \
  source=/content/ball_dataset/valid/images save=True


Streaming output truncated to the last 5000 lines.
 extracting: /content/ball_dataset/train/labels/a3-mp4_snapshot_00-07_-2022-10-17_19-57-36-_jpg.rf.917262146b512c7d7729f26723caf82f.txt  
 extracting: /content/ball_dataset/train/labels/a3-mp4_snapshot_00-08_-2022-10-17_19-56-50-_jpg.rf.eee1c63c5d415bab7bc3154f5c8a91c3.txt  
 extracting: /content/ball_dataset/train/labels/a3-mp4_snapshot_00-08_-2022-10-17_19-57-02-_jpg.rf.2bdb026da02626fbe8d655f136641f90.txt  
 extracting: /content/ball_dataset/train/labels/a30447a5ee9cb57b_jpg.rf.2dc999e694c8c71dab926497dd4258ab.txt  
 extracting: /content/ball_dataset/train/labels/a332d3d4eafa1787_jpg.rf.36147703b4724a76732091563bb7400b.txt  
 extracting: /content/ball_dataset/train/labels/a3d07b50d8ab98cc_jpg.rf.8ddf2baee16fbea9f78880957df19dee.txt  
 extracting: /content/ball_dataset/train/labels/a3d9d708db8473b7_jpg.rf.22478f075c0e4b98b1dec2f65d8dae3d.txt  
 extracting: /content/ball_dataset/train/labels/a4-mp4_snapshot_00-00_-2022-10-17_19-58-08-

In [ ]:

!unzip "/content/drive/MyDrive/volley_actions_dataset.zip" -d /content/actions_dataset

import os
from pathlib import Path
import shutil

def filter_spike_only(source_split):
    label_dir = Path(f"/content/actions_dataset/{source_split}/labels")
    image_dir = Path(f"/content/actions_dataset/{source_split}/images")

    target_label_dir = Path(f"/content/spike_dataset/{source_split}/labels")
    target_image_dir = Path(f"/content/spike_dataset/{source_split}/images")
    target_label_dir.mkdir(parents=True, exist_ok=True)
    target_image_dir.mkdir(parents=True, exist_ok=True)

    for label_file in label_dir.glob("*.txt"):
        with open(label_file, "r") as f:
            lines = f.readlines()
        spike_lines = [line for line in lines if line.startswith("4 ")]  # 4 = spike class
        if spike_lines:
            with open(target_label_dir / label_file.name, "w") as f_out:
                f_out.writelines(spike_lines)

            img_path = image_dir / label_file.with_suffix(".jpg").name
            if img_path.exists():
                shutil.copy(img_path, target_image_dir / img_path.name)

filter_spike_only("train")
filter_spike_only("valid")

with open("/content/spike_dataset/data.yaml", "w") as f:
    f.write("""train: /content/spike_dataset/train/images
val: /content/spike_dataset/valid/images
nc: 1
names: ["spike"]
""")

!yolo task=detect mode=train model=yolov8n.pt \
  data=/content/spike_dataset/data.yaml \
  epochs=20 imgsz=640


In [ ]:
from IPython.display import Image, display
import glob

for img_path in sorted(glob.glob("runs/detect/predict/*.jpg"))[:5]:
    display(Image(filename=img_path))
